In [1]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import albumentations as A
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from LookGenerator.datasets.segmentation_generator_dataset import SegGenDataset
from LookGenerator.networks.encoder_decoder import EncoderDecoder
from LookGenerator.networks.losses import FocalLoss
from LookGenerator.networks.segmentation import UNet
from LookGenerator.networks.trainer import Trainer
from LookGenerator.networks.utils import load_model
from LookGenerator.networks_training.utils import check_path_and_creat
from torchvision.transforms import ToTensor, ToPILImage
from LookGenerator.networks.clothes_feature_extractor import ClothAutoencoder

In [2]:
toten = ToTensor()
topil = ToPILImage()

In [3]:
augment = A.Resize(256,192)
batch_size_train = 32
batch_size_val = 16
pin_memory = True
num_workers = 0

In [4]:
train_dataset = SegGenDataset(image_dir=r'C:\Users\Даша\кто\мусор\zalando-hd-resized\train',
                              #r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\train",
                        augment=augment)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size_train, pin_memory=pin_memory, num_workers=num_workers)

In [13]:
for input_, target in train_dataloader:
    print(input_.shape)
    # input_ = input_[0]
    # pic = transforms.ToPILImage()(input_[:3])
    # pic.show()
    # for i in range(3,11):
    #     pic = transforms.ToPILImage()(input_[i])
    #     pic.show()
    break

torch.Size([32, 11, 256, 192])


In [6]:
val_dataset = SegGenDataset(image_dir=r'C:\Users\Даша\кто\мусор\zalando-hd-resized\test',
                            #r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\test",
                        augment=augment)
val_dataloader = DataLoader(train_dataset, batch_size=batch_size_val, pin_memory=pin_memory, num_workers=num_workers)

In [7]:
# clothes_feature_extractor = ClothAutoencoder(
#     in_channels=3,
#     out_channels=3,
#     features=(8, 16, 32, 64),
#     latent_dim_size=128,
#     encoder_activation_func=nn.LeakyReLU(),
#     decoder_activation_func=nn.ReLU()
# )
# clothes_feature_extractor = load_model(clothes_feature_extractor, r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\autoDegradation\weights\testClothes_L1Loss_4features\epoch_39.pt")

In [14]:
model = UNet(in_channels=11,out_channels=3)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = FocalLoss()
print(device)

cpu


In [9]:
save_directory = r'C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\segGen\weights\baseParam'
check_path_and_creat(save_directory)

RuntimeError: 

In [15]:
trainer = Trainer(
    model_=model, optimizer=optimizer, criterion=criterion,
    device=device, save_directory=save_directory, save_step=1, verbose=True
)

In [16]:
epoch_num=40

In [17]:
trainer.train(train_dataloader=train_dataloader, val_dataloader=val_dataloader, epoch_num=epoch_num)

start time 10-06-2023 08:46


  0%|          | 0/364 [00:00<?, ?it/s]Exception ignored in: <generator object tqdm.__iter__ at 0x000002134EDCA120>
Traceback (most recent call last):
  File "C:\Users\Даша\PycharmProjects\SMBackEnd\venv\lib\site-packages\tqdm\std.py", line 1196, in __iter__
KeyboardInterrupt: 
  0%|          | 0/364 [00:21<?, ?it/s]

KeyboardInterrupt



In [ ]:
trainer.draw_history_plots()

In [ ]:
trainer.save_history_plots(save_directory)

わたしの最後はあなたがいい

In [ ]:
model = load_model(model, r'C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\segmentationMulty\weights\testMulty_out_12_6features_noGauss\epoch_39.pt')

In [ ]:
test_dataset = SegGenDataset(image_dir=r"",
                        augment=augment)
test_dataloader = DataLoader(test_dataset, batch_size=1)

In [ ]:
for input_, target in test_dataloader:
    result = model(input_)

    result = result.squueze(0)

    fig, axes = plt.subplots(2, 3)

    for i in range(3):
        axes[0,i].imshow(np.array(topil(result[i])), cmap = 'gray')
        axes[1,i]. imshow(np.array(topil(target[i])), cmap = 'gray')
    break